Your name: Huy Tran

Your student ID number: 33259590

Shared link to this notebook: https://github.com/hvtran21/cs446_p3

# Programming Assignment 3 (P3) for COMPSCI 446, Search Engines


This project is focused on indexing and query processing using a small collection of documents. Before starting, be sure to review Chapter 5 in the textbook, especially section 5.3, which covers inverted indexes, and section 5.6 (with an emphasis on 5.6.1), which discusses index construction. For the query processing part, sections 5.7.1 and 5.7.2, as well as sections 7.1.1, 7.2.2, and 7.3.1, will be particularly useful.

Your task is to implement `index_run(file, query_list, trecrun_file)` method. This method will accomplish 3 things:
* [Document Parsing](#document-parsing) : Parse the collection of documents in
* [Query Processing](#query-processing) : Read and process a list of queries from `query_list`.
* [TREC Run Formatting](#output-formatting) : Output the results in TREC run format (as in P2) to `trecrun_file`.

Only the implementation of `index_run(file, query_list, trecrun_file)` will be graded. However, we will provide a skeleton of helper methods you can implement for this project. We will not test the provided helper functions or even verify that they exist. (Though please be certain that those code cells run, even if you do that by making an empty function.)

Here are a list of provided files that will be loaded into your Google Drive for use in this notebook:

* _sciam.json.gz_, a compressed document collection comprising of a number of stories from issues of Scientific American from the 1800s. They are all from Project Gutenberg’s collection. The “books” were automatically broken into stories and each story is a “document” for purposes of this project. A URL was also automatically included that takes you to a nice version of the story if you’re interested in looking at it (or its images). These stories have been partially vetted for objectionable material (sensibilities in the 1800s were different than they are today); if you find something that we overlooked, please let us know.

* _P3train.tsv_, a file containing a list of sample queries to run as input to your program for testing.

* _P3train.trecrun_ is a file containing a sample, trecrun-formatted output for _P3train.tsv_.

* _queries.tsv_, a file containing a list of queries that you will be graded on for your submission; we are not providing sample output for them.

##0. Prerequisites

In [2]:
version = 2 # DO NOT MODIFY. If notebook does not match with autograder version, many tests are likely to fail!!

###Link your Google Drive
Execute the following to connect to Google Drive (you will be prompted repeatedly for access to your Google Drive; please give it permission) and download copies of the sample files listed above. You should not need to make any modifications to the code, though if you want to use a slightly different path in Google Drive, you can modify the appropriate drive_path value. (The autograder will not use your Google Drive and will fail if you insert code that explicitly uses the Google Drive.)

In [3]:
import os
import string
import gzip
import re

from collections import Counter

try:
    from google.colab import drive
    in_colab = True
except ImportError:
    in_colab = False


# You are more than welcome to code some helper functions.
# But do note that we are only grading functions that are coded in the template files.


# Connect to Google Drive and download copies of the sample files listed above.
# Please allow the access to your Google Drive or the following dataset loader will fail.
# (The autograder will not use your Google Drive.)
if in_colab:
  drive.mount("/content/drive/") ## DO NOT MODIFY THIS LINE
  data_path = "/content/drive/MyDrive/CS 446 Search Engines/p3" ## CHANGE TO YOUR OWN FOLDER ON GOOGLE DRIVE
else:
  data_path = "./data/"  ## DO NOT MODIFY THIS LINE. CHANGING THIS LINE WOULD RESULT IN FAIL OF AUTOGRADER TESTS

assert os.path.exists(data_path), "Change data_path to a valid and existing file path in your google drive!"

### Load the provided files

In [4]:
import urllib.request
import gzip
import json
import sys
import os
from pathlib import Path

def load_file(file_path: str, gz_zip: bool = True, is_json: bool=False) -> list[str]:
    """
    Load strings from the text or gzip file. Remember to strip newline if necessary!

    Args:
        file_path: the location of file we want to analyze on.
        gz_zip: true if the file is gzipped, false otherwise

    Returns: an array of string loaded from the text or gzip file.
    """
    webloc = "https://cs.umass.edu/~allan/cs446/"


    data_info = Path(data_path)
    if not data_info.exists() or not data_info.is_dir():
      print(f"Google folder \"{data_path}\" is not present or not a folder. Nothing will work from here.")
      return []

    local_google_drive_path = os.path.join(data_path,file_path)
    local_file = Path(local_google_drive_path)
    if local_file.is_file():
        print(f"File \"{file_path}\" already exists, not downloading.")
    else:
        print(f"Cannot find \"{file_path}\" so downloading it")
        urllib.request.urlretrieve(webloc + file_path, local_google_drive_path)
        print("Done")

    results = []
    if not gz_zip:
        f = open(local_google_drive_path, "r", encoding="utf-8")

    else:
        # Read compressed file (opened in text mode since that's what we use)
        f = gzip.open(local_google_drive_path, "rt", encoding="utf-8")

    # Pull in the contents of the file and return it
    if is_json:
      results = json.load(f)['corpus']
    else:
      results = [line.strip("\n") for line in f.readlines() if line]

    if f:
      f.close()

    return results



# path to documents and stopwords
documents_path = "sciam.json.gz" # path to gzip file that contains documents
P3_train_path = "P3train.tsv" # Path to p1 train dataset
P3_trecrun_path = "P3train.trecrun"  # path to the file that contains stopwords

documents = load_file(documents_path, is_json=True)
trecrun  = load_file(P3_trecrun_path, gz_zip = False)    # Downloading sample queries file to folder, these are \t seperated files (Not in json format)
queries  = load_file(P3_train_path, gz_zip = False)      # Download sample queries output file to folder, these are \t seperated files (Not in json format)

File "sciam.json.gz" already exists, not downloading.
File "P3train.trecrun" already exists, not downloading.
File "P3train.tsv" already exists, not downloading.


## Dataset

The dataset is stories from Scientific American issues from the 1800s. The data has been tokenized and stemmed (not stopped) and made available as sciam.json.gz. (You will probably want to look at it uncompressed, but be sure that your code handles the compressed version only.) The stories were extracted from 25 issues of the magazine. If you're curious, you can look at the original magazine issues at https://www.gutenberg.org/ebooks/bookshelf/256.

This dataset has been preprocessed by stripping out punctuation and applying the Krovetz Stemmer (not Porter). No stopwords have been removed. Note that this means that the tokenization and stemming steps (as in P1) have been performed and, because stemming has already happened, that _you will not use a stopword list_. (Plus, of course, this will allow you to support queries with conjunction words -- e.g., the three words “clumps and keys” in order!)

An example story is below. The text element is a actually single line in the input file but is presented wrapped here for ease of presentation.  The tokens are always separated by space characters, meaning that you will essentially be using a space tokenizer!

```
{  "article" : "8952",
   "storyID" : "8952-id_18",
   "storynum" : "id_18",
   "url" : "https://www.gutenberg.org/cache/epub/8952/pg8952-images.html#id_18",
   "text" : "invention patent in england by americacompile from the journal of
   the commissioner of patentprovisional protection for six month3 201 sewing
   machine h a house bridgeport conn november 4 1869 3 211 bore tool alexander
   allen new york city november 5 1869 3 215 mode of and device foe secure
   stair rod h uhry new york city november 6 1869 3 229 transportation of
   letters parcel and other freight by atmospheric pressure and in apparatus
   connected therewith a e beach stratford conn november 9 1869 3 303 reload
   cartridge shell r j gatl indianapolis ind november 16 1869 3 342 wooden
   pavement i hayward and j f paul boston mass november 20 1869 3 358 machinery
   for distribute type o l brown boston mass november 20 1869 3 219 weigh
   machine m kennedy new york city november 10 1869 3 260 bran duster w huntley
   and a babcock silver creek n y november 12 1869 3 339 railway carriage e
   robbin cincinnati ohio november 19 1869 3 341 revolving battery gun r j gatl
   indianapolis ind nov 19 1869 3 360 sash fastener s l loomi south byron n y
   november 20 1869 3 363 magnetic machine and magnet j burrough jr newark n j
   november 20 1869  "  
}
```

This story is from article number 8952 published in Scientific American, Volume 22, No. 1, January 1, 1870. The story has an internal number of `id_18`: the storyID combines the article number and the story number to yield `8952-id_18`. The URL will get you to that story directly. If you scroll to the top you'll see the article's date.

For purposes of P3, only the storyID and the text are important. You may find the others helpful, particularly the URL if you want to decipher some of these documents.


<a name="document-parsing"></a>
## 1. Parse the collection of documents in `file` to build an in-memory index.

To implement the first component of `index_run()`, we want to read in the tokenized and stemmed document collection provided and build an inverted index. Here are some things you will probably want to take into account.

- The documents are stored using JSON, so you'll need to read them in that way.
Python has a JSON library in the standard library which should suffice; we discourage you from spending the time to parse JSON on your own!
- Because of the pre-processing we have done for you, term vectors should be constructed by splitting the text field's value by spaces (the regex \\s+), and ignoring blank strings. You've already shown you can tokenize, stop, and stem, so we're making it easier for this project.
 - Note that the processing in this case did not squeeze apostrophes out: instead, they were treated as word separators. That means that the word `quaker's` has resulted in tokens `quaker` and also `s`. Similarly, you'll find `t` (from `don't`) and `ve` (from `I've`) and so on as tokens. You will handle them as if they were normal words: do not remove them! That way the phrase query `"quaker s"` (see Part II) should find the word `quaker's` (and also `quaker s'truth` if by some weird chance that appeared in the stories).
- Note that the index includes Unicode characters, but you should not worry about them even if they create odd tokens. Tokens are whatever is separated by a whitespace.
- You need to build a simple inverted index with count and positional information: a combination of Figure 5.4 (p.134) and Figure 5.5 (p.135) in the textbook. (Depending on how you store things, the count may come along for free with the position information.) While you may store your index to disk to enable reuse, it is not required: an in-memory only index is sufficient for the project. (Your decision may be made partly by how long it takes to process the full collection in your implementation. It should be fast.)

- Here are some recommendations regarding the data structures you will use behind these functions to implement the indea. You can use whatever data structures you feel are appropriate to implement these functcions, however, you may find these ideas helpful. We will not explicitly test your data structures.
  - We recommend having the data storage behind your inverted index being of a form similar to **Dict[str, List[Posting]]**. Other data structures are plausible but are unlikely to provide any particular value for this project.
 - You might define a **PostingList** class that provides the **List[Posting]** datatype. Mostly that would help abstract away the implementation of the list.
 - **Posting** might be a **DocId** and **Positions** pair, so that a dictionary is used to find inverted lists and the inverted list is a list of "Postings." It might also include the count of postings in that document.
 - **Positions** might be a **List[int]** or **array** of integers
 - **For the sake of efficiency (string comparisons are slow)** and memory usage, we encourage you to encode document numbers as integers, not using the *storyID* textual field. Otherwise you will not be able to answer analysis questions about compression. However, note that you will still need to be able to convert a document number back into its corresponding *storyID* for the TREC run output file. For example: getStoryID(1) would be “8951-id_1” since that's the first story in *sciam.json.gz*.

- We suggest you think ahead (see [processing queries](#query-processing) below) and ensure your index can access the vocabulary, term counts, document counts and other statistics that you will need to perform the query evaluation activities. Thinking this through now will save some headaches later.

 - You will be writing code to handle arbitrary (but simple) queries for phrases and terms. That means your code will benefit from being general in how it handles them. You will likely find it best to have a method that allows any number of terms and/or phrases and does not depend on knowing what those terms and/or phrases will be or even how many there will be. Here are a couple of tips or ideas that may or may not help your thinking:
   - You will be implementing scoring functions and models that produce scores that are not just integers. Put some time into it now (read ahead!) to ensure that your inverted index provides everything that is needed for those operations -- e.g., the number of terms, the number of documents, and so on. What do you think you might need to store and where would it make sense to store it?
   - Would it benefit you to define helper functions for processes such as counting phrases and terms?
   - Can you implement terms as phrases of length 1? That is, have one function that handles phrases and single terms that doesn't care whether there are 1 or 32 terms in the phrase. That provides a cleaner abstraction.

Below is the stub of a helper function that can be implemented to build and return an inverted index for `index_run()`. You are not required to use this, and it will not be graded.

In [27]:
import math
import sys
import gzip
import json
import os

# postings can be doc id: term count, list[positions]
# how inverted list should be implemented:
#    term: Posting()
#        -> where object posting can be defined as shown above
# need to make it easy to get statistics, like term counts, document counts so on and so forth.
# need to use json parsing library (python standard) to make parsing file easier
# -> list of dictionaries i believe or something.

"""
a posting is mapping from a single term that gives information that:
    -> total times it occurs in the entire corpus
    -> a list of documents that the term occurs in
        -> the positions of which the term occurs in that document
        -> the number of time that term occurs in that document

"""
def load_json(file_path: str, is_json: bool) -> None:
    f = gzip.open(file_path, "rt", encoding="utf-8")

    if is_json:
        results = json.load(f)['corpus']
    else:
        results = [line.strip("\n") for line in f.readlines() if line]
    
    return results

class Posting:
    def __init__(self):
        self.total_term_count = 0
        self.doc_to_id = {}     # doc number: storyID
        self.id_to_doc = {}     # storyID: doc number
        self.doc_to_term = {}   # doc_id: [term_count, [positions]]   

    def add_story_id(self, document_number: int, storyID: str) -> None:
        self.doc_to_id[document_number] = storyID
        self.id_to_doc[storyID] = document_number
    
    # doc_id will correspond to the document number
    def add_doc_id_info(self, doc_id: int, term_count: int, positions: list[int]) -> None:
        if doc_id in self.doc_to_term.keys(): # if we want to add a new term and the doc_id already exists, we just want to update the information
            new_count = term_count + self.doc_to_term[doc_id][0] # first position is term count
            self.doc_to_term[doc_id][1].extend(positions)
            self.doc_to_term.update({doc_id: [new_count, self.doc_to_term[doc_id][1]]})

        else:
            self.doc_to_term[doc_id] = [term_count, positions]

    def get_document_occurence(self): # returns the number of documents that this term occurs in
        return len(self.doc_to_term.keys())
        
def build_inverted_index(file):
    inverted_index = {}

    for i, dictionary in enumerate(file):
        document = dictionary.get('text')
        positions = {}

        for position, word in enumerate(document.split()): # obtains word: list(type: int, contains: zero-indexed positions)
            if word:
                positions.setdefault(word, []).append(position)
            
        for word, positions in positions.items():
            if word not in inverted_index:
                new_posting = Posting()
                new_posting.add_story_id(i, dictionary.get("storyID"))
                new_posting.add_doc_id_info(i, len(positions), positions)
                inverted_index[word] = new_posting
            else:
                inverted_index[word].add_story_id(i, dictionary.get("storyID"))
                inverted_index[word].add_doc_id_info(i, len(positions), positions)

            inverted_index[word].total_term_count += len(positions)
    return inverted_index

# Example usage
d = load_json("./data/sciam.json.gz", is_json=True)
inverted_index = build_inverted_index(d)


### 1.1 Debugging the Index

We recommend that you consider a helper method to print out some information to help you debug your index. There are other ways to do this, but we're setting things up to ensure you can do it this way if you like. We will not test this function or even verify that it exists. (Though please make sure the cell does not trigger an error.)

For example, you might print out some information that will be helpful for verifying your index overall for some array of terms. It could first print the number of documents, the number of unique terms that are in the collection, and the total number of term occurrences. Then, for each term listed, it could print out one line with the term, the number of documents containing that term, and the total number of occurrences of that term.

Here is some sample output that you should get for this data using this collection, but since we will not actually test your code (it is for debugging only), the actual format does not matter.

```
debug_inverted_index(inverted_index, ["and","the", "elephant","science"], showTerms=False)

976 docs, 27217 terms, 1185999 occurrences
and 956 docs, 34896 occurrences
the 966 docs, 96151 occurrences
elephant 7 docs, 18 occurrences
science 112 docs, 251 occurrences

```

You might also consider creating something that prints out parts of the inverted list so you can see what's there. For example, you might have the argument “showTerms” that prints out the inverted list for the listed terms. For example, you might show something like this:


```
debug_inverted_index(inverted_index, ["elephant", "science"], showTerms=True)

**** elephant - 7 docs, 18 occurrences
  at {8952-id_12 [259]} {38481-art31 [317, 438, 464]} {38482-art25 [3, 24, 47]} {8391-id_13 [5, 35, 80, 106, 350, 1014, 1225, 1466]} {8504-id_30 [38]} {24322-art09 [647]} {15193-art17 [396]}
**** science - 112 docs, 251 occurrences
  at {8951-id_9 [35]} {8951-id_21 [171]} {8952-id_15 [1561]} {8952-id_30 [40, 69, 120]} {8952-id_31 [5, 108, 134, 172, 406, 429, 485, 493, 559, 623, 739, 780, 956, 1063, 1072, 1126, 1183, 1391, 1684, 1693, 1941, 2015, 2172, 2222, 2302, 2343, 2392, 2404, 2551, 2564]} {8952-id_33 [196, 244, 524]} {8952-id_44 [112]} {8952-id_46 [18]} {8952-id_69 [9595, 15195]} {19180-art22 [1645]} {19180-art28 [90, 152]} {19180-art33 [131, 389]} {19180-art35 [277, 302, 565, 701, 852, 926]} {19180-art41 [152]} {19180-art53 [4613, 4625, 10221, 11314, 11326, 17166]} {19406-art48 [329, 554, 862, 1537]} {19406-art31 [5]} {19406-art44 [502, 726]} {19406-art68 [892]}
```

(For reasons of space, the output for `science` is truncated and the output for `and` as well as `the` is not shown at all.)

Reminder, providing these debug functions is entirely optional. The autograder will not test whether you have any code that does that. However, you are likely to find something like this helpful as you debug your code -- e.g., it gives you a list of the documents that your query processing should return for simple one-word queries. Also, it's fair for the teaching staff to ask you for something comparable when you post a question that appears like it might be the result of incorrect indexing.

In [28]:
def debug_inverted_index(inverted_index, terms, showTerms=True):
    """
        Print out debugging information for the index

        Args:
            inverted_index: the inverted index data structure.
            terms: an array of strings, representing the terms to debug
            showTerms: defaults to true, will print out inverted list instead of # of docs and occurrences for each term

        Returns: n/a
      """
    for term in terms:
        if term in inverted_index.keys():
            posting = inverted_index[term]
            print(term, posting.total_term_count, "occurrences", posting.get_document_occurence(), "docs")
            

# Example usage
debug_inverted_index(inverted_index, ["and", "the", "elephant","science"], showTerms=False)


and 34896 occurrences 956 docs
the 96151 occurrences 966 docs
elephant 18 occurrences 7 docs
science 251 occurrences 112 docs


<a name="query-processing"></a>
## 2. Read and process a list of queries from `query_list`

The second component of `index_run()` requires you to run a sequence of queries that are provided in *queries.tsv*. You will want to have read the appropriate parts of Chapter 7 in the text. Recall that *sciam.json.gz* is the set of documents you index and *queries.tsv* is a list of queries to run. We are also providing *P3train.tsv* and *P3train.trecrun* which are a set of sample queries and their expected output, respectively.

The list of queries (in *P3train.tsv* and in *queries.tsv*) comprises lines with at least three fields separated by tabs (tsv = tab separated values) in the format:

> queryType queryName wordPhrase1 wordPhrase2 … wordPhraseN

where:

- *queryType* is one of the following, indicating what type of query you will be processing (case does not matter, so AND, and, And, aNd, and so on, are all the same):
 - **AND** means a Boolean query that is the AND of all of the *wordPhrases* listed.
 - **OR** means a Boolean query that is the OR of all of the *wordPhrases* listed.
 - **QL** means you will run a query likelihood algorithm with the *wordPhrases* as the query. QL is discussed starting in 7.3.1 of the textbook. You are asked to implement the version that uses Dirichlet smoothing, discussion of which starts on page 258 of both versions of the textbook. Use μ=300.
 - **BM25** means you will use the BM25 algorithm with the *wordPhrases* being the query. BM25 scoring is described starting on page 250 of the PDF or printed textbook. Please use these values for your scoring: k_1=1.8, k2=5, b=0.75.
 - **TF** means you will calculate the raw term frequency of *wordPhrase* for each story.
 - **DF** means you will calculate the document frequency of each *wordPhrase* listed (yes, even if it is a phrase). Your output (see below) will show the *wordPhrase* as if it were a document and the DF for the score. Rank will be ignored.
- *queryName* is a provided name for this query (e.g., "query1" or "12354" or "smelt"). The first column of every output line will be this *queryName* (for this query).
- *wordPhraseK* are the words/phrases that you need to find using your index.
 - If there are multiple words (separated by spaces) in a *WordPhrase*, it means a phrase and that those words must occur adjacent in an article and in that given order. Although phrases are often indicated by quotation marks elsewhere, they are _not_ used here.
 - Note that there will always be at least one *wordPhrase* and there is no theoretical limit on the value of N.


###**Handling phrases**

If a *wordPhrase* contains one or more spaces, it is a phrase. (Recall that the *wordPhrases* are separated by tabs, so there can be spaces within them.) A phrase requires a match with two or more adjacent words in each document.

For QL and BM25 (and DF), you need to know the number of times a phrase occurs in a document. (For Boolean you just have to know that number is at least one for a given document.) To do that, you'll have to actually do the equivalent of running that phrase as if it were a query. Remember that you are only interested in phrases, where the words occur next to each other in the order given; you do not need to handle things like “within 5 words” or “in the same sentence.”

Although TF and DF and essentially index-probing "query" types, you must allow multiple *wordPhrase* occurrences as in all of the others and expect them to be phrases or single words. For TF, you will be scoring and ranking the documents by the TF count (with ties broken appropriately). For DF, your output will display the *wordPhrase* where the document would normally be, the DF value where the score would be, and the rank will be ignored.



<a name="output-formatting"></a>
###**Output format**

The output of your program will be a file called *output.trecrun* containing a number of lines per query in *queries.tsv*. You are going to use the "TREC run format" as follows. For each query, you will display its ranked list using exactly six columns per line. White space (spaces or tabs) is used to separate columns. The width of the columns in the format is not important, but it is important to have exactly six columns per line with at least one space between the columns (note that no queryName or storyID contains a space). The format is,

> *queryName* skip *storyID* rank score *yourUsername*

where

- the first column is the *queryName* from the file *queries.tsv*
- the second column is currently unused and should always be `skip`.
- the third column is the value of the *storyID* JSON field of this ranked document. (For the DF query type, it should be the *wordPhrase*.)
- the fourth column is the rank of the document retrieved. You will list things sorted by rank within the query, with the highest score having rank 1 and there being no ties (even though there might be ties in the scores; break ties by sorting by *storyID*). The rank will be ignored for the DF type, though you must provide a value.)
- the fifth column shows the score (integer or floating point) that generated the ranking. This score must be in descending (non-increasing) order. The score for a Boolean query will always be 1.0. The score for TF and DF query types should be the term frequency or document frequency, respectively.
- the sixth column is called the "run tag" and is traditionally a unique identifier for you and for the method used. In this case, just use your SPIRE username (e.g., "allan" for the professor). Please do not turn in results with "username" (or "allan") as the identifier.


Here is an example of what this might look like (the information is made up; see sample output for real results). It aligns the columns to make them easier to read; you do not have to do that. For example,
```
Q1    8951-id_20           1    1.000   username
Q1    19406-art03          2    0.500   username
Q1    38481-artnq13        3    0.333   username
Q1    21081-article44-11   4    0.250   username
…
Q1    13443-elec1          237  0.003   username
Qtwo  38481-artnq13        1    0.998   username
Qdf   some phrase          1    10      username
…
```
**Note that all queries are printed to _output.trecrun_** (the filename passed into eval below), **with all matching stories per query. For QL, BM25, and TF, a story matches a ranked query if it contains at least one of the query terms; it matches a Boolean query if it satisfies the Boolean expression.** If any QL, BM25, AND, OR query happens to retrieve zero documents, there will be no lines in the output file for that query. If any TF query has 0 term frequency for some document, that document should be omitted from your output. Any term frequency greater than zero for any document should otherwise be printed out to *output.trecrun*.

If any *wordPhrase* in a DF query has 0 document frequency, there will be no line in the output file for that *wordPhrase*. Also note that for the DF measure, the *wordPhrase* instances are independent of each other: they are not processed as a group as they are for the other items. Just output one line per *wordPhrase* word or phrase.


### 2.1 Running Boolean queries
Your program will be expected to handle queries that find words, phrases (two or more adjacent words), or combinations of both. The queries will return the complete set of *storyID*s that match the Boolean query. Obviously there will never be more than the total number of stories in the collection.

There are no scores other than 1.0/0.0 calculated, so sort the matching documents (all of which have a score of 1.0) by the *storyID* string (as a string, ignoring that it might look like a number in some ways -- i.e., “11-x” comes before “2-x”) and print out the list that way, increasing the rank from 1 through the number that match.

Below are sample helper methods that you can use to help implement boolean query processing for `index_run()`. However, remember that you are not required to use this method, and will not be graded on the method itself.

In [ ]:
def filter_postings(wordPhrase: str, single_word: bool) -> dict[(str, int): list[int]]:
    """
    wordPhrase is excpeted to include a single space with > 1 word
    for example: united states, unicorn pony
    """
    word_info = {} # {(word: str, doc_id: int): positions: list[int]}
    split_phrase = [word for word in wordPhrase.split() if word != ""] # split off of spaces and don't include empty strings
    doc_id_lst = []

    if all(True if word in inverted_index.keys() else False for word in split_phrase):
        for word in split_phrase:
            tmp = [] # holds doc_ids of which the current word of iteration occurs in
            for doc_id in inverted_index[word].doc_to_term.keys():
                tmp.append(doc_id)
            doc_id_lst.append(tmp) # list of lists, each individually represents what docs contain which term, where the indice is the key

        combined_lst = []
        if not single_word:
            for i in range(1, len(split_phrase)):
                cur_lst = doc_id_lst[i]     # docs that element 1 occurs in
                prev_lst = doc_id_lst[i-1]  # docs that element 2 occur in 
                combined_lst.append(set(cur_lst).intersection(set(prev_lst))) # combined_lst gives use documents that these elements occur in
                
            combined_lst = set.intersection(*combined_lst)
            if len(combined_lst) == 0:
                return None # if intersection results elements sharing no common documents, then the phrase cannot be returned properly.
        else:
            combined_lst = tmp
        
        for word in split_phrase:
            for doc_id, items in inverted_index[word].doc_to_term.items():
                if doc_id in combined_lst:
                    word_info[(word, doc_id)] = items[1]
        return word_info

def intersect(inverted_index, wordPhrases):
    """
      Evaluate an AND boolean query over all wordPhrases in the query

      Args:
          inverted_index: the inverted index data structure.
          wordPhrases: a list of words or phrases to search for.

      Returns: an array of tuples, where the first value is the doc_id, and the second is the score
    """
    
    results = []

    wordPhrase_lst = wordPhrases.split("\t")
    filtered_postings = filter_postings(wordPhrase_lst[0], single_word=False) # dict[(word, doc_id): positions[int]]
    
    # below we can use a similar method, but need to use the above implementation.
    # if len(check_phrase) != len(wordPhrases): return results # if these lengths do not match, the intersect must fail, one can assume all doc_id pair values will be 0.0
    # doc_lst = []
    # for phrase in check_phrase:
    #     doc_lst.append([(doc_id, inverted_index[phrase].doc_to_id[doc_id]) for doc_id in inverted_index[phrase].doc_to_term.keys()])  # for each term, generates a list of doc_ids that contain this term
    # intersect_doc_ids = set(doc_lst[0]).intersection(*doc_lst[1:])   # filters out the doc_ids that don't contain all query values
    # for pair in intersect_doc_ids:     # any documents that doesn't have a generateted value is assumed to be zero
    #     results.append((pair[1], 1.0))
    
    return results

test_phrase = "improvement in hull\tnitro glycerin professor"
print(intersect(inverted_index, test_phrase))

{('improvement', 0): [0], ('improvement', 5): [16989], ('improvement', 22): [984, 1211, 1231, 2493, 2683, 3439, 3511, 3687], ('improvement', 95): [7991, 12266], ('improvement', 168): [799], ('improvement', 217): [2777, 5836], ('improvement', 276): [4988, 5155, 9051, 10577, 11108, 11625, 12311, 12639], ('improvement', 336): [2694, 3931, 4021, 5258, 6749, 15336, 15393, 15952, 16009], ('improvement', 449): [4159, 5191, 5740, 5873, 5913, 9140, 9561], ('improvement', 489): [1668, 1778, 2440], ('improvement', 741): [89], ('in', 0): [1, 93, 95, 120, 134, 222, 268, 275, 284, 450, 475, 489], ('in', 5): [118, 219, 252, 393, 431, 492, 527, 547, 578, 645, 717, 814, 902, 1039, 1076, 1102, 1159, 1163, 1194, 1209, 1262, 1295, 1394, 1726, 2022, 2038, 2041, 2081, 2129, 2152, 2167, 2220, 2296, 2373, 2391, 2496, 2519, 2573, 2622, 2674, 2704, 2757, 2776, 2841, 2888, 2970, 2980, 2998, 3014, 3055, 3141, 3319, 3336, 3374, 3632, 3640, 3673, 3755, 3788, 3810, 3833, 4203, 4266, 4379, 4432, 4477, 4541, 4619, 470

In [8]:
def union(inverted_index, wordPhrases):
    """
      Evaluate an OR boolean query over all wordPhrases in the query

      Args:
          inverted_index: the inverted index data structure.
          wordPhrases: a list of words or phrases to search for.

      Returns: a set of tuples, where the first value is the doc_id, and the second is the score
    """

    results = set()

    #########
    ##
    ## Implement the function here
    ##
    #########

    return results

### 2.2 Running QL and BM25 queries

The QL and BM25 approaches calculate a score that is intended to reflect the probability of relevance in some way. That score produces the ranking.

For QL, the score is the product of a number of small numbers (probabilities), which runs the risk of creating arithmetic underflow. Represent probabilities as the log (**natural log**) of the probabilities and sum them. That means that the score for these ranking runs will be negative, since the log of a number less than one (e.g., a probability) will be negative. Sorting in reverse order by the negative scores will put higher probabilities at the start of the list as you want.

For these ranking queries, only print documents at ranks 1 through 100, inclusive, unless the query matches fewer than 100 stories, in which case list everything retrieved -- **that is, any story that includes at least one of the query terms**. If there are documents with the same score, break the tie by sorting those by the storyID textual field's value (as a string, ignoring that it looks like a number in some ways -- i.e., “11-x” comes before “2-x”).

Below are sample helper methods that you can use to help implement QL and BM25 query processing for `index_run()`. They copy the constant values that were listed above for ease of reference. However, you are not required to use this method, and will not be graded on it.

In [9]:
def query_likelihood(inverted_index, wordPhrases):
    """
      Evaluate a QL query over all wordPhrases

      Args:
          inverted_index: the inverted index data structure.
          wordPhrases: a list of words or phrases to search for.

      Returns: a set of tuples, where the first value is the doc_id, and the second is the score
    """
    # need to keep track of number of words per document, maybe edit data structure
    ranking = []
    mu = 300  # Dirichlet smoothing parameter

    #########
    ##
    ## Implement the function here
    ##
    #########

    return ranking

In [10]:
def bm25(inverted_index, words):
    """
      Perform a bm25 query over all words in the query

      Args:
          inverted_index: the inverted index data structure.
          words: a list of words or phrases to search for.

      Returns: an array of tuples, where the first value is the doc_id, and the second is the score
    """
    # need tf, document length, average document length, N (# of docs in collection)
    results = []
    k1 = 1.8
    k2 = 5.0
    b = 0.75

    #########
    ##
    ## Implement the function here
    ##
    #########

    return results

###2.3 Running TF and DF queries

TF and DF calculate a score that is intended to reflect the raw term frequency or document frequency, respectively. That score produces the ranking.

For TF, the score for a document is the raw term frequency, or the number of times a *wordPhrase* occurs in the document. For any TF query, we will calculate the raw term frequency across all documents, and rank according to the raw term frequency. Ties should be broken according to the *storyID* (as in AND and OR). If the TF is 0 for any document, do not list that document in the output.

For DF, the output is a bit different. The "storyID" field will be replaced by the *wordPhrase*, the score will be the document frequency for that *wordPhrase*, and the rank will be ignored. If the DF is 0 for the given *wordPhrase*, we will not print anything in *output.trecrun* for that *wordPhrase*. **NOTE: If the _wordPhrase_ is a phrase -- i.e., it contains spaces -- replace all of the spaces with underscores.** That is because the trecrun file format does not allow spaces in docids and we want to be consistent with that format. So `DF qname united<tab>states<tab>united states` woudl result in one line for `united`, another for `states`, and one for the phrase that would look liked `united_states`. Note that if the query were actually `DF qname united_states` the output would look the same (`united_states`) but the DF count would almost certainly be different.

Below are sample helper methods that you can use to help implement TF and DF query processing for `index_run()`. However, you are not required to use this method, and will not be graded on it.

In [11]:
def tf(inverted_index, doc_id, words):
    """
      Calculate the number of times a term or phrase appears in a document

      Args:
          inverted_index: the inverted index data structure.
          doc_id: the document we want to calculate the term frequency for.
          term: the term or phrase we want to calculate the frequency for.

      Returns: an integer representing the number of times the term appears in doc_id.
    """

    results = []
    #########
    ##
    ## Implement the function here
    ##
    #########

    return results

In [12]:
def df(inverted_index, wordPhrase):
    """
      Calculate the number of documents a term or phrase appears in
      NOTE that this sample only handles one wordPhrase at at time and so only returns
      a single number. This is primarily to highlight that it operates differently
      than the others that are ranking documents.

      Args:
          inverted_index: the inverted index data structure.
          term: the term or phrase we want to calculate the frequency for.

      Returns: an integer representing the number of documents a term or phrase appears in.
    """

    doc_freq = 0

    #########
    ##
    ## Implement the function here
    ##
    #########


    return doc_freq

## 3. Implement `run_queries()`

This is the only method that you will be graded on, which should create your inverted index based on the file named by `documents` (such as *sciam.json.gz*), parse and evaluate all queries in `queries_list` (as in *queries.tsv*), and print out the results to a file named by `trecrun_file` (for example, *output.trecrun*).

In [13]:
def run_queries(documents, query_list, trecrun_file):
    """
      Create an inverted index, parse a query file, run them on the inverted index,
      and print out the results in trecrun format.

      Args:
          documents: the name of a file containing the documents to index
          query_list: the name of the file containing queries to run
          trecrun_file: the name of the file that will be created to contain the
                        output of running the queries

      Returns: void
    """

    #########
    ##
    ## Implement the function here
    ##
    #########

    return

# Example usage (generates P3train.myrun to compare to P3train.trecrun)
run_queries("sciam.json.gz", "P3train.tsv", "P3train.myrun")

### 3.1 Sample queries and answers

We are providing *P3train.tsv* that you can use to try out your system. We are also providing you with the expected output for those queries as *P3train.trecrun*. Your code will be tested on those same queries plus a number of other queries, so please try other possibilities: do not assume that because your code handles the training queries your code is perfect. You may use the below method, *print_file()*, to print out *P3train.trecrun* to compare your results manually. You might want to create a variation that also reads in your output and does a comparison to find where things are different. However, this part will not be graded, and is optional to implement. As always, just make sure that it does not trigger an error in the notebook.

In [14]:
def print_file(file):
    """
      Print out the contents of a file

      Args:
          file_path: the path to the file

      Returns: n/a
    """

    #########
    ##
    ## Implement the function here
    ##
    #########

    print()

# Example usage
print_file(trecrun)

In [15]:
def print_queryfile(file_path):
    """
      Print out the contents of a file

      Args:
          file_path: the path to the file

      Returns: n/a
    """
    #########
    ##
    ## Implement the function here
    ##
    #########

# Example usage
print_queryfile(queries)

## 4. Analysis Questions

Answer the questions below. You will probably find it easiest to answer some of them if you engineer one of your functions above to display these numbers after indexing and then you can copy them below.

Unlike the way P1 and P2 were implemented, these analysis questions are designed to be handled by the autograder, though they will be hidden "tests" so you will not have confirmation that you got them right until after grading is completed.

**4.1**. What is the average length of a story in the *sciam* collection? What is the shortest story (and how short it is)? What is the longest story (and how long is it)? Note that for this project, "short" and "long" are measured by the number of tokens, not the number of characters.

In [16]:
# Provide the correct values for each of these "variables" as described above. Do not provide any other code.

def analysis_q1():

  averageStoryLength =
  averageStoryStoryId = ""
  shortestStoryLength =
  longestStoryStoryId = ""
  longestStoryLength =

  return [averageStoryLength, averageStoryStoryId, shortestStoryLength, longestStoryStoryId, longestStoryLength]


SyntaxError: invalid syntax (2584224492.py, line 5)

**4.2**. What word occurs in the most stories and how many stories does it occur in? What word has the largest number of occurrences and how many does it have?


In [ ]:
# Provide the correct values for each of these "variables" as described above. Do not provide any other code.

def analysis_q2():

  termOccurringInMostStories = ""
  numberOfStoriesItOccursIn =
  termOccurringMostFrequently = ""
  numberOfTimesItOccurs =

  return [termOccurringInMostStories, numberOfStoriesItOccursIn, termOccurringMostFrequently, numberOfTimesItOccurs]


**4.3.** How many unique words are there in this collection? How many of them occur only once? What percent is that? Is that what you would expect? We are not asking you to return this, but to increase the chance that you have the right answer, you might think about why or why not?


In [ ]:
# Provide the correct values for each of these "variables" as described above. Do not provide any other code.

def analysis_q3():

  numberUniqueWords =
  numberWordsOccurringOnce =
  percentOfWordsThatOccurOnce = # be sure this is a percent (without the %) and not a fraction

  return [numberUniqueWords, numberWordsOccurringOnce, percentOfWordsThatOccurOnce]

**4.4**. Your training queries have two queries that are roughly about the scientific american supplement. Suppose that you wanted to judge stories for relevance using a pooling strategy that takes the top 100 documents from each of those two queries. How many unique documents will you be judging? What if you only considered the top 20? Suppose you had a budget that allowed you to judge at most 30 documents. How deeply could you go into the two queries for judging to get 30 judged, no more, no less?




>Enter your response here



In [ ]:
# Provide the correct values for each of these "variables" as described above. Do not provide any other code.

def analysis_q4():

  numberUniqueJudged100 =
  numberUniqueJudged20 =
  poolDepthFor30 =

  return [numberUniqueJudged100, numberUniqueJudged20, poolDepthFor30]